In [48]:
from feature.base_dataset import *
from feature.after_dataset import *
from glob import glob

In [49]:
train_input_list = sorted(glob('./data/train_input/*.csv'))
train_target_list = sorted(glob('./data/train_target/*.csv'))

test_input_list = sorted(glob('./data/test_input/*.csv'))
test_target_list = sorted(glob('./data/test_target/*.csv'))

In [71]:
train['내부온도관측치'].var()

8.858872931501214

In [72]:
def group_statistics(df):  
    '''
    변수의 Case별로 DAT을 기준으로 groupby 하여 대표값 설정  
    mean, median, std, var, quantile
    '''
    df_a = pd.DataFrame()

    for i, v in enumerate(df["Case"].unique()):
        train_old = df[df['Case']==v]
        train_old = train_old.groupby(['DAT']).var().reset_index()
        df_a = pd.concat([df_a, train_old])
        df_a = df_a.drop(['일간누적분무량', '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량',
                          'predicted_weight_g', 'Case', 'DAT', 'obs_time'], axis=1)
    return df_a

In [73]:
train = make_dataset(train_input_list, train_target_list)
test = make_dataset(test_input_list, test_target_list)

train = time_value(train)
test = time_value(test)

train = limit_range(train)
test = limit_range(test)

train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

test['predicted_weight_g'] = 0

train = col_cumsum(train, "시간당분무량", "일간누적분무량")
train = col_cumsum(train, "시간당백색광량", "일간누적백색광량")
train = col_cumsum(train, "시간당적색광량", "일간누적적색광량")
train = col_cumsum(train, "시간당청색광량", "일간누적청색광량")
train = col_cumsum(train, "시간당총광량", "일간누적총광량")

c:\mb_job\project\Competition\Lettuce-Growth-Environment-Prediction\feature\base_dataset.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum


In [74]:
train = group_statistics(train)
test = group_statistics(test)

In [76]:
train.to_csv('train_var.csv', index=False)
test.to_csv('test_var.csv', index=False)